In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt

In [28]:
data_folder = './Data'

In [29]:
#Defining our Cleaning function that can handle both 1--Column and 12-Column file

In [30]:
def clean_trade_file(file_path, country_code):
    try:
        df_raw = pd.read_excel(file_path, skiprows=7)
        df_raw = df_raw[df_raw.iloc[:, 0].notna()]
        df_raw = df_raw[~df_raw.iloc[:, 0].astype(str).str.contains("TOTAL", na=False, case=False)]

        num_cols = df_raw.shape[1]

        if num_cols == 10:
            df_raw.columns = [
                "Code", "Product", "Value_2023", "Trade_Balance_2023",
                "Growth_2019_2023", "Growth_2022_2023", "World_Growth",
                "World_Import_Rank", "Avg_Distance_km", "Concentration"
            ]
            # Add missing columns as NaN
            df_raw["World_Import_Share"] = None
            df_raw["Avg_Tariff"] = None

        elif num_cols == 12:
            df_raw.columns = [
                "Code", "Product", "Value_2023", "Trade_Balance_2023",
                "Growth_2019_2023", "Growth_2022_2023", "World_Growth",
                "World_Import_Share", "World_Import_Rank", "Avg_Distance_km",
                "Concentration", "Avg_Tariff"
            ]
        else:
            print(f"Skipping {country_code}: unexpected number of columns")
            return None

        df_raw["Code"] = df_raw["Code"].astype(str).str.replace("'", "").str.strip()
        df_raw["Country"] = country_code
        df_raw["Direction"] = "Import"

        return df_raw

    except Exception as e:
        print(f"Failed to clean {country_code}: {e}")
        return None


In [31]:
#Loop through all the country folders and use the above cleaning function.

In [32]:
master_data = []

# Get all folders (country codes)
country_folders = [f for f in os.listdir(data_folder) if os.path.isdir(os.path.join(data_folder, f))]

for country_code in country_folders:
    folder_path = os.path.join(data_folder, country_code)
    files = [f for f in os.listdir(folder_path) if f.endswith("&AU IM.xlsx")]
    if not files:
        continue

    file_path = os.path.join(folder_path, files[0])
    df = clean_trade_file(file_path, country_code)

    if df is not None:
        master_data.append(df)


In [33]:
#Combining into a maste dataframe

In [34]:
master_df = pd.concat(master_data, ignore_index=True)

In [35]:
#The below csv files is out master data file. However, ther are two coulms World_Import_Share and Avg_Tarrif which are empty for most 
#of the countries. This is beacus these columns were only present in the 12-column Excel files (e.g., from UAE, KSA, etc.). And the rest
#are left as empty. 
# But during modeling and clusturing we will remove these 

In [36]:
master_df.to_csv("cleaned_trade_master.csv", index=False)

In [3]:
#Now we have cleaned master dataset we are going to go for Exploratory Data Analysis 